In [1]:
# Stacking 示例（处理类别不平衡后，meta 使用 LogisticRegression C=0.5）
# 基准：两个 Logistic(C=1)、一个 CatBoost、一个 LightGBM，最终 meta: Logistic(C=0.5)
# 说明：在 fit 时使用 SMOTE 做重采样（目标占比约 20-30%），并使用 5 折 CV 评估 AUC。

In [2]:
# Imports
import os
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 路径（根据你的仓库结构调整）
ROOT = r'd:\Competition\数科统模'
TRAIN_PATH = os.path.join(ROOT, 'data', 'data(processed)', 'train.csv')
TEST_PATH = os.path.join(ROOT, 'data', 'data(processed)', 'test.csv')

# 读取数据（若文件路径不同，请调整）
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print('train shape:', train.shape)
print('test shape:', test.shape)

# 假设目标列名为 'target'，若不同请修改下面的变量名
TARGET = 'target'
if TARGET not in train.columns:
    raise ValueError(f"目标列 '{TARGET}' 未在 train 文件中找到，请检查 train.csv 列名。")

# 简单检查目标分布
print(train[TARGET].value_counts(normalize=True))

train shape: (500, 24)
test shape: (2000, 23)
target
0    0.98
1    0.02
Name: proportion, dtype: float64


In [4]:
# 准备特征矩阵 X 和标签 y（自动剔除 id 和 target）
drop_cols = [c for c in ['id', 'ID', 'index'] if c in train.columns]
X = train.drop(columns=drop_cols + [TARGET], errors='ignore')
y = train[TARGET].values
X_test = test.drop(columns=drop_cols, errors='ignore')

print('features:', X.shape[1])

# 定义基学习器
lr1 = LogisticRegression(solver='liblinear', penalty='l2', C=1.0, max_iter=10000, random_state=42)
lr2 = LogisticRegression(solver='liblinear', penalty='l2', C=0.2, max_iter=10000, random_state=0)
cat = CatBoostClassifier(verbose=0, random_state=42)
lgb = LGBMClassifier(random_state=42, n_jobs=-1)

# Stacking 元学习器（meta）
meta = LogisticRegression(solver='liblinear', penalty='l2', C=0.5, max_iter=1000, random_state=42)

estimators = [
    ('lr1', lr1),
    ('lr2', lr2),
    ('cat', cat),
    ('lgb', lgb)
]

stack = StackingClassifier(estimators=estimators, final_estimator=meta, cv=5, n_jobs=-1, passthrough=False)

# 使用 imblearn Pipeline，把填充、标准化、SMOTE 和 stacking 串起来
pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(sampling_strategy=0.25, random_state=42)),  # minority/majority ~= 0.25 -> minority frac ~20%
    ('stack', stack)
])

features: 22


In [5]:
# 交叉验证评估（AUC）
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=1)
print('CV AUC scores:', scores)
print('Mean AUC: %.6f (+/- %.4f)' % (scores.mean(), scores.std()))

CV AUC scores: [0.31632653 0.54591837 0.55102041 0.35714286 0.97959184]
Mean AUC: 0.550000 (+/- 0.2351)


In [6]:
# 在全部训练数据上训练并对测试集做预测（概率）
pipeline.fit(X, y)
if hasattr(pipeline, 'predict_proba') or hasattr(pipeline.named_steps['stack'], 'predict_proba'):
    proba = pipeline.predict_proba(X_test)[:, 1]
else:
    # fallback: decision_function -> scale to [0,1] via sigmoid
    from scipy.special import expit
    dec = pipeline.decision_function(X_test)
    proba = expit(dec)

# 构建提交文件（假设 test 有 'id' 列或使用索引）
id_col = None
for c in ['id', 'ID', 'index']:
    if c in test.columns:
        id_col = c
        break
if id_col is None:
    submission = pd.DataFrame({'id': np.arange(len(proba)), 'target': proba})
else:
    submission = pd.DataFrame({id_col: test[id_col].values, 'target': proba})

out_dir = os.path.join(ROOT, 'submit')
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'stacking_logit_meta_logitC05_submission.csv')
submission.to_csv(out_path, index=False)
print('Saved submission to', out_path)

Saved submission to d:\Competition\数科统模\submit\stacking_logit_meta_logitC05_submission.csv


## 说明与可调参数
- 我们在 pipeline 中使用 SMOTE(sampling_strategy=0.25) 将少数类扩充到约 20%（minority/majority=0.25 -> minority fraction ≈ 0.20）。可根据需要调整为 0.2-0.33。
- 基学习器: 两个 Logistic(C=1)、CatBoost、LightGBM。若希望让两个 Logistic 不完全相同，可再调整 penalty、solver 或添加不同特征子集。
- meta 模型: Logistic(C=0.5)。如果需要做超参搜索，可以对 final_estimator 的 C 使用 GridSearch（注意计算成本）。
- 评估: 使用 5 折 Stratified CV 计算 AUC。StackingClassifier 内部也会使用 cv=5 来生成 meta 特征。

要运行: 按顺序运行每个单元格。若缺少依赖库 (imblearn, catboost, lightgbm)，请先安装。